### Kafka code to connect to Coin API and extract three exchanges trade for Bitcoin Cryptocurency

In [2]:
import websocket
import json
from kafka import KafkaProducer

# Configuration
api_key = "1AFF18CB-7F59-4F04-85C8-AB6A8BA01260"  # Use your valid CoinAPI key
websocket_url = "wss://ws.coinapi.io/v1/"
kafka_bootstrap_servers = 'localhost:9092'  # Use the same address as your successful test

# Initialize a Kafka Producer
producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_servers],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

# Define callback functions for WebSocketApp
def on_message(ws, message):
    message_data = json.loads(message)
    print(f"Message received: {message}")

    # Determine the target Kafka topic based on the symbol_id
    target_topic = None
    if "BITSTAMP_SPOT_BTC_USD" in message_data["symbol_id"]:
        target_topic = "bitstamp_btc_usd"
    elif "BITFINEX_SPOT_BTC_USD" in message_data["symbol_id"]:
        target_topic = "bitfinex_btc_usd"
    elif "KRAKEN_SPOT_BTC_USD" in message_data["symbol_id"]:
        target_topic = "kraken_btc_usd"
    
    # Publish the message to the appropriate Kafka topic
    if target_topic:
        producer.send(target_topic, value=message_data)
        print(f"Published to Kafka topic: {target_topic}")
    else:
        print("Symbol ID not recognized or not configured for routing.")

def on_error(ws, error):
    print(f"Error occurred: {error}")

def on_close(ws, close_status_code, close_message):
    print("WebSocket connection closed.")

def on_open(ws):
    subscription_message = {
        "type": "hello",
        "apikey": api_key,
        "heartbeat": False,
        "subscribe_data_type": ["trade"],
        "subscribe_filter_symbol_id": [
            "BITSTAMP_SPOT_BTC_USD",
            "BITFINEX_SPOT_BTC_USD",
            "KRAKEN_SPOT_BTC_USD"
        ]
    }
    ws.send(json.dumps(subscription_message))
    print("Subscribed to trade updates for BTC/USD pairs from multiple exchanges.")

# Create and run the WebSocketApp
if __name__ == "__main__":
    ws_app = websocket.WebSocketApp(websocket_url,
                                    on_open=on_open,
                                    on_message=on_message,
                                    on_error=on_error,
                                    on_close=on_close)
    ws_app.run_forever()


Subscribed to trade updates for BTC/USD pairs from multiple exchanges.
Message received: {"time_exchange":"2024-04-27T22:40:55.0280000Z","time_coinapi":"2024-04-27T22:40:55.1689688Z","uuid":"30ee2614-196c-4ed1-80dc-848dd5ac7ec9","price":63409,"size":0.00075569,"taker_side":"SELL","symbol_id":"BITFINEX_SPOT_BTC_USD","sequence":82242,"type":"trade"}
Published to Kafka topic: bitfinex_btc_usd
Message received: {"time_exchange":"2024-04-27T22:40:55.1050000Z","time_coinapi":"2024-04-27T22:40:55.2447453Z","uuid":"5cb2beaa-39f7-4724-a7fa-69080a1b870c","price":63409,"size":0.01456176,"taker_side":"SELL","symbol_id":"BITFINEX_SPOT_BTC_USD","sequence":82243,"type":"trade"}
Published to Kafka topic: bitfinex_btc_usd
Message received: {"time_exchange":"2024-04-27T22:40:56.7234010Z","time_coinapi":"2024-04-27T22:40:56.8441758Z","uuid":"d9e4197e-8cf4-4fb7-9d57-89e331298737","price":63241.20000,"size":0.00014455,"taker_side":"BUY","symbol_id":"KRAKEN_SPOT_BTC_USD","sequence":99299,"type":"trade"}
Pub